In [3]:
import cx_Oracle
import os
import matplotlib.pyplot as plt
from DB import DAO

# 로그인

In [2]:
from DB import DAO
def login_ck(mlogin,mid,mpw='') :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기    
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    if mlogin == 'D' :
        cursor.execute(f"""SELECT mb_id, mb_signup_type, mb_type FROM tbl_member 
                        where mb_signup_type='D' and mb_id='{mid}'and mb_pw='{mpw}'""")     
    else :
        cursor.execute(f"SELECT mb_id, mb_signup_type, mb_type FROM tbl_member where mb_signup_type='{mlogin}' and mb_id='{mid}'")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if len(data):
        data = data[0]
        session['user_id'] =  data[0]
        session['login_by'] = data[1]
        session['manager'] = data[2]
        return redirect(url_for('login_info', info='001777'))
    else :
        return redirect(url_for('login_info', info='001666'))

def select_all(mlogin,mid) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""SELECT mb_name, mb_id, mb_phone, mb_email, mb_signup_type, mb_type FROM tbl_member 
                    where mb_signup_type='{mlogin}' and mb_id='{mid}'""")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data:
        data = data[0]
        return data
    else :
        return render_template('/404')
def id_ck(ID) :
    result = False
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""SELECT count(*) FROM tbl_member 
                    where mb_id='{mid}'""")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data:
        data = data[0]
        if data == 0 :
            result = True
    
    return result

# 커뮤니티

In [3]:
def call_paging(now, total) :
    index = 0
    if now % 5 == 0 :
        index = 5
    else :
        index = now % 5
    start = now - index + 1
    end = min(total, now-index+5)
    
    paging = list(range(start,end + 1,1))
    
    if now > 5 :
        paging = ['<']+paging
    
    if total > end :
        paging += ['>']
    return paging
def select_content_list(now,content_type,sort_by='latest',selector='a', search='') :
    if sort_by == 'likes' :
        sort_by = 'commu_like desc'
    elif sort_by == 'views' :
        sort_by = 'commu_views desc'
    else :
        sort_by = 'commu_update2 desc'
    
    query = ''
    
    if selector == 't' :
        query = f"t1.commu_title like '%{search}%'"
    elif selector == 'w' :
        query = f"t1.mb_id like '%{search}%'"
    elif selector == "c" :
        query = f"t1.commu_content like '%{search}%'"
    else :
        query = f"(t1.commu_title like '%{search}%' or t1.commu_content like '%{search}%' or t1.mb_id like '%{search}%')"
        
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""
        select *  from 
        (select commu_seq, rownum as commu_index, commu_title, mb_id, commu_update, commu_views, commu_like
        from (select t1.commu_seq, t1.commu_title, t1.mb_id, t1.commu_like, t1.commu_views, t1.commu_update, to_char(t1.commu_update, 'YYYY-MM-DD HH24:MI:SS') as commu_update2 
        from tbl_community t1 where t1.commu_type='{content_type}' and {query} order by {sort_by},t1.commu_seq desc))
        where commu_index > ({now}-1)*20 and commu_index <= {now}*20
            """)
    data = cursor.fetchall()
    content_list = data
    cursor.execute(f"""
        select ceil(count(*)/20) from tbl_community where commu_type = '{content_type}'
            """)
    data = cursor.fetchall()
    total = int(data[0][0])
    cursor.close()
    conn.close()
    
    return content_list, total

def call_content(commu_seq) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""
        select commu_title, commu_content, mb_id, commu_update, commu_like, commu_views, commu_seq from tbl_community where commu_seq={commu_seq}
            """)
    data = cursor.fetchall()
    if data :
        data = data[0]
    else :
        data = ()
    cursor.close()
    conn.close()
    return data

def update_view(commu_seq) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"update tbl_community set commu_views = commu_views + 1 where commu_seq={commu_seq}")
    cursor.execute("commit")
    cursor.close()
    conn.close()
    
def update_content(commu_seq, commu_title, commu_content) :
#     os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"update tbl_community set (commu_title, commu_content, commu_update) = ('{commu_title}', '{commu_content}',sysdate) where commu_seq={commu_seq}")
    cursor.execute("commit")
    cursor.close()
    conn.close()
    return redirect(url_for('community'))

# 마이 페이지

In [4]:
def cal_delay(time) :
    time = float(time)
    temp_delay = ''
    if round(time) != 0 :
        temp_delay += str(round(time))+'일'
    time *= 24
    if round(time) % 24 != 0 :
        temp_delay += str(round(time) % 24)+'시간'
    time *= 60
    if round(time) % 60 != 0 :
        temp_delay += str(round(time) % 60)+'분'
    time *= 60
    if round(time) % 60 != 0 :
        temp_delay += str(round(time) % 60)+'초'
    return time_delay + '경과'

def sensor_condition(data) :
    for kit in [('DHT11_HUMI',['건조', 50, '정상', 80, '과습']), ('DHT11_TEMP',['저온', 20, '정상',  30, '고온'])] :
        try :
            dergree = data['kit'][kit[0]]
            if degree < kit[1][1] :
                data['kit'][kit[0]] = [degree,kit[1][0]]
            elif degree < kit[1][3] :
                data['kit'][kit[0]] = [degree,kit[1][2]]
            else :
                data['kit'][kit[0]] = [degree,kit[1][4]]
            if data['kit'][kit[0]][-1] != '정상' :
                cursor.execute(f"""
                select (SYSDATE - t1.sv_date) AS DELAY
                from tbl_kit_sensoring t1
                inner join tbl_kit t2
                on t1.kit_seq=t2.kit_seq
                where t2.cage_seq = {data['cage_seq']} and t2.kit_name = '{kit[0]}'
                and t1.sv_date = 
                (select MAX(sv_date) from tbl_kit_sensoring where kit_seq = t1.kit_seq AND SV_VALUE > {kit[1][1]} AND SV_VALUE < {kit[1][3]})

                union all

                select (sysdate - kit_inst_date) as DELAY from tbl_kit where cage_seq = {data['cage_seq']} and kit_name = '{kit[0]}'
                and not exists
                (select (SYSDATE - t1.sv_date) AS DELAY
                from tbl_kit_sensoring t1
                inner join tbl_kit t2
                on t1.kit_seq=t2.kit_seq
                where t2.cage_seq = {data['cage_seq']} and t2.kit_name = '{kit[0]}'
                and t1.sv_date = 
                (select MAX(sv_date) from tbl_kit_sensoring where kit_seq = t1.kit_seq AND SV_VALUE > {kit[1][1]} AND SV_VALUE < {kit[1][3]}))
                """)
                data1 = cursor.fetchall()
                time = float(data1[0][0])
                data['kit'][kit[0]] += [cal_delay(time)]
        except :
            pass
    for kit in ['HEATING_PAD','HUMIDIFIER','COOLING_PAN8*8','COOLING_PAN8*8'] :
        try :
            if int(date['kit'][kit]) == 1 :
                date['kit'][kit] = '켜짐'
            elif int(date['kit'][kit]) == 0 :
                date['kit'][kit] = '꺼짐'
            else :
                date['kit'][kit] = '에러' 
        except :
            pass
    return data
    
def call_cage_list(mb_id) :  # 케이지 번호, 이름, 애완동물 종류, 애완동물 이름
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT T3.CAGE_SEQ, T3.CAGE_NAME, LISTAGG(T3.PET_TYPE,'/') WITHIN GROUP (ORDER BY T3.PET_TYPE) AS PET_TYPE, LISTAGG(T3.PET_NAME,'/') WITHIN GROUP (ORDER BY T3.PET_NAME) AS PET_NAME
        FROM (SELECT T1.CAGE_SEQ, T1.CAGE_NAME,T2.PET_TYPE, LISTAGG(T2.PET_NAME,',') WITHIN GROUP (ORDER BY T2.PET_NAME) AS PET_NAME, T1.MB_ID
        FROM TBL_CAGE T1, TBL_PET T2
        WHERE T1.CAGE_SEQ = T2.CAGE_SEQ
        GROUP BY T1.CAGE_SEQ, T1.CAGE_NAME, T2.PET_TYPE, T1.MB_ID
        ORDER BY T1.CAGE_SEQ, COUNT(T2.PET_TYPE) DESC) T3
        WHERE T3.MB_ID = '{mb_id}'
        GROUP BY T3.CAGE_SEQ, T3.CAGE_NAME
    """)
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data :
        cage_list = []
        for cage in data :
            cage_info = [cage[0],cage[1]]
            for i in range(2,4) :
                if len(cage[i]) > 10 :
                    comma_index = 0
                    slash_index = 0
                    if ',' in cage[i][:11] :
                        comma_index = cage[i][:11].rindex(',')
                    if '/' in cage[i][:11] :
                        slash_index = cage[i][:11].rindex('/')
                    index = max(comma_index, slash_index)
                    cage_info.append(cage[i][:index+1]+'...')
                else :
                    cage_info.append(cage[i])
            cage_list.append(cage_info)
        return cage_list
    else :
        return []

def call_cage_seq(mb_id, cage_seq) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"select cage_name from tbl_cage where cage_seq = {cage_seq} and mb_id = '{mb_id}'")
    data = cursor.fetchall()
    if data != [] :
        data = {'cage_seq': cage_seq, 'cage_name' : data[0][0]}
        cursor.execute(f"""
            select t1.kit_name, t2.sv_value
            from (select t3.kit_seq, max(t3.sv_date) as sv_date, t4.kit_name 
            from tbl_kit_sensoring t3, tbl_kit t4 
            where t4.kit_seq=t3.kit_seq and t4.cage_seq = {cage_seq}
            group by t3.kit_seq, t4.kit_name) t1
            inner join tbl_kit_sensoring t2 
            on t1.kit_seq = t2.kit_seq
            where t1.sv_date = t2.sv_date
        """)
        data1 = cursor.fetchall()
        data['kit'] = dict(data1)
        cursor.execute(f"""
            select distinct t3.pet_type, listagg(t3.pet_info, '\n') WITHIN GROUP (ORDER BY t3.pet_info) AS PET_INFO
            FROM (select PET_TYPE, (PET_NAME ||'('||
            (CASE
            WHEN PET_GENDER = 'f' THEN '♀'
            WHEN PET_GENDER = 'm' THEN '♂'
            ELSE '?'
            END)
            ||')') as pet_info
            from tbl_pet WHERE CAGE_SEQ=2) T3 GROUP BY T3.PET_TYPE
            """)
        data2 = cursor.fetchall()
        data['pet'] = data2
        result = sensor_condition(data)
        cursor.close()
        conn.close()
        return result
    else :
        cursor.close()
        conn.close()
        return redirect(url_for('mypage_info', info='365666'))
def load_graph(today,  mb_id, cage_seq) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""
        select trunc(avg(sv_value),2), to_char(median(sv_date), 'AM HH:MI') from tbl_kit_sensoring t1 inner join tbl_kit t2 on t1.kit_seq = t2.kit_seq where t2.kit_name = 'DHT11_TEMP' and to_char(sv_date, 'YYYY-MM-DD') = '{today}' GROUP BY to_char(sv_date, 'AM HH:MI')
            """)

    data = cursor.fetchall()
    temp = [[a[0] for a in data],[a[1] for a in data]]

    cursor.execute(f"""
        select trunc(avg(sv_value),2), to_char(median(sv_date), 'AM HH:MI') from tbl_kit_sensoring t1 inner join tbl_kit t2 on t1.kit_seq = t2.kit_seq where t2.kit_name = 'DHT11_HUMI' and to_char(sv_date, 'YYYY-MM-DD') = '{today}' GROUP BY to_char(sv_date, 'AM HH:MI')
            """)

    data = cursor.fetchall()
    humi = [[a[0] for a in data],[a[1] for a in data]]

    cursor.execute(f"""
        select trunc(avg(sv_value),2), to_char(median(sv_date), 'AM HH:MI') from tbl_kit_sensoring t1 inner join tbl_kit t2 on t1.kit_seq = t2.kit_seq where t2.kit_name = 'PAD_TEMP' and to_char(sv_date, 'YYYY-MM-DD') = '{today}' GROUP BY to_char(sv_date, 'AM HH:MI')
            """)

    data = cursor.fetchall()
    pad_temp = [[a[0] for a in data],[a[1] for a in data]]


    plt.figure(figsize=(20,8))
    plt.plot(range(0, 6,1), temp[0], c='red', marker='*')
    plt.plot(range(0, 6,1), humi[0], c='blue', marker='*')
    plt.plot(range(0, 6,1), pad_temp[0], c='orange', marker='*')
    plt.xticks(range(0, 6,1), labels=temp[1], rotation=45)
    plt.xlim(0, 5)
    plt.ylim(10, 60)
    plt.grid()
    plt.title(f'Today_Sensoring({today})')
    plt.savefig(f'./static/img/{today}_{mb_id}_{cage_seq}.png')
    cursor.close()
    conn.close()

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for
from markupsafe import escape
from datetime import timedelta, datetime
import random


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

token, info, cage_list, cage_lifo = '', '', '', ''

######### 메인 화면 #########
@app.route('/') # home 
def index() :
    return render_template('home.html', session=session)

@app.route('/<info>')  # home + 이전 페이지 결과
def index_info(info) :
    if len(session) == 0 and info == '001777':
        # session 없는데 로그인 성공 떴을 때
        info = '705000'
    return render_template('home.html', session=session)


######### 회원 가입 #########
@app.route('/signup') # 회원 가입
def signup() :
    if len(session) :
        return redirect(url_for('signup_info', info='705001'))
    return render_template('signup.html', session=session)

@app.route('/signup/confirm') # 회원 가입 + 정보 확인
def confirm_id() :
    return render_template('confirm_id.html')

@app.route('/signup/confirm/<info>') # 회원 가입 + 정보 확인 + 결과
def confirm_info(info) :
    return render_template('confirm_id.html')

@app.route('/signup/confirm/result', methods=['GET']) # 회원 가입 + 정보 확인 + 입력값 체크
def signup_result() :
    if request.method == 'GET' :
        mb_id = request.args['mb_id']
        return id_ck(mb_id)


######### 로그인 #########
@app.route('/login') # 로그인
def login() :
    if len(session) :
        return redirect(url_for('login_info', info='705001'))
    return render_template('login.html')

@app.route('/login/<info>') #로그인 + 이전 페이지 결과
def login_info(info) :
    return render_template('login_result.html')
    

@app.route('/login/result', methods=['POST']) # 로그인 + 입력값 체크
def login_result() :
    if request.method == 'POST' :
        mlogin = request.form['mlogin']
        mid = request.form['mid']
        mpw = ''
        if mlogin == 'N' : 
            token = request.form['token']
        elif mlogin == 'D' :
            mpw = request.form['mpw']
        return login_ck(mlogin,mid,mpw)

    
######### 로그아웃 #########
@app.route('/logout') # 로그아웃
def logout() :
    if len(session) :
        session.pop('user_id', None)
        session.pop('login_by', None)
        session.pop('manager', None)
        if len(session) == 0 :
            info = '000777'
        else :
            info = '000666'
    else :
        info = "705000"
    return redirect(url_for('index_info', info=info))



######### 소개 페이지 #########
@app.route('/diamond/team') # 팀소개
def diamond() :
    return render_template('diamond_team.html', session=session)

@app.route('/diamond/shop') # 쇼핑몰
def shop() :
    return render_template('diamond_shop.html', session=session)

@app.route('/diamond/contact') # 오시는 길
def contact() :
    return render_template('diamond_contact.html', session=session)




############ 게시판 페이지 ########

@app.route('/community/<commu_type>') # 게시판 첫화면
def community(commu_type) :
    return redirect(url_for(f'community_detail', commu_type=commu_type, now=1, sort_by='latest'))

@app.route('/community/<commu_type>/<sort_by>/n<now>') #게시판 with 페이징, 정렬
def community_detail(commu_type, now, sort_by) :    
    if commu_type in ['free', 'qna', 'faq'] :
        commu_t = ''
        
        if commu_type == 'free' :
            commu_t = 'F'
        elif commu_type == 'qna' :
            commu_t = 'Q'
        elif commu_type == 'faq' :
            commu_t = 'U'
        content_list, total = select_content_list(now, commu_t, sort_by)
        paging = call_paging(int(now), int(total))
        return render_template(f'community_{commu_type}.html', sort_by=sort_by, search='', paging=paging, content_list=content_list, session=session)
    else :
        return 
    
def community_detail_info(commu_type, now, sort_by, info) :    
    if commu_type in ['free', 'qna', 'faq'] :
        commu_t = ''
        
        if commu_type == 'free' :
            commu_t = 'F'
        elif commu_type == 'qna' :
            commu_t = 'Q'
        elif commu_type == 'faq' :
            commu_t = 'U'
        content_list, total = select_content_list(now, commu_t, sort_by)
        paging = call_paging(int(now), int(total))
        return render_template(f'community_{commu_type}.html', info=info, sort_by=sort_by, search='', paging=paging, content_list=content_list, session=session)
    else :
        return 
    

@app.route('/community/<commu_type>/i<info>') # 게시판 + 알림창
def community_info(commu_type, info) :
    return redirect(url_for('community_detail_info', commu_type=commu_type, now=1, sort_by='latest', info=info))

@app.route('/community/<commu_type>/<sort_by>/n<now>/<selector>_<search>') # 게시판 with 페이징, 정렬, 검색어
def community_withS(commu_type, now, sort_by, selector, search) :
    if commu_type in ['free', 'qna', 'faq'] :
        commu_t = ''
        
        if commu_type == 'free' :
            commu_t = 'F'
        elif commu_type == 'qna' :
            commu_t = 'Q'
        elif commu_type == 'faq' :
            commu_t = 'U'
        content_list, total = select_content_list(now, commu_t, sort_by, selector, search)
        paging = call_paging(int(now), int(total))
        return render_template('community_free.html', sort_by=sort_by, search=search, paging=paging, content_list=content_list, session=session)
    else :
        pass
        
@app.route('/community/c<commu_seq>') # 게시판 글 보는 페이지
def see_content(commu_seq) :
    update_view(commu_seq)
    content_info = call_content(commu_seq)
    return render_template('community_free_content.html', content_info=content_info, sessiom=session)

@app.route('/community/write/t_<ctype>') # 게시판 글 등록 페이지
def write_content(ctype) :
    if len(session) :
        return render_template('')
    else :
        return redirect(url_for('community_info', info='705000')

@app.route('/community/update/c<commu_seq>') # 게시판 글 수정 페이지
def update_content(commu_seq) :
    if len(session) :
        content_info = call_content(commu_seq)
        if session['user_id'] == content_info[2] :
            return render_template('community_content_update.html', content_info=content_info, session=session)
        else :
            return redirect(url_for('community_info', info='123666')
    else :
        return redirect(url_for('community_info', info='705000'))

@app.route('/community/update/result', methods=['POST'])
def update_result() :
    mb_id = request.form['mb_id']
    if session['user_id'] == mb_id :
        commu_seq = request.form['commu_seq']
        commu_title = request.form['commu_title']
        commu_content = request.form['commu_content']
        update_content(commu_seq, commu_title, commu_content)
    else :
        return redirect(url_for('community_info', info='123666'))
    
    
########## 마이페이지 ##########
@app.route('/mypage/cage') #사육장 목록
def mypage() :
    if len(session) :
        cage_list = call_cage_list(session['user_id'])
        return render_template('mypage_cage.html', session=session, cage_list=cage_list)
    else :
        return redirect(url_for_for('index_info', info='705000')) 
    
@app.route('/mypage/cage/<info>') #사육장 + 정보 확인
def mypage_info(info) :
    if len(session) :
        cage_list = cage_list(session['user_id'])
        return render_template('mypage_cage.html', session=session, cage_list=cage_list, info=info)
    else :
        return redirect(url_for_for('index_info', info='705000')) 
    
@app.route('/mypage/cage/n<cage_seq>')  # 사육장 자세히 보기
def see_cage(cage_seq) :
    today = '2022-08-30'
    # today = datetime.today()
    if len(session) :
        cage_info = call_cage_seq(session['user_id'], int(cage_seq))
        load_graph(today,session['user_id'], cage_seq)
        keys = cage_info['kit'].keys()
        return render_template('mypage_cage_show.html', session=session, cage_info=cage_info, keys=keys, today=today)
    else :
        return redirect(url_for_for('index_info', info='705000')) 
    
@app.route('/img/<img_src>')  # 사진 크게 보기
def big_img(img_src) :
    return render_template('big_img.html')

######### 에러 처리 #########
@app.errorhandler(404)
def page_not_found(e):
    return render_template("404.html"), 404    
    
@app.errorhandler(500)
def page_not_found(e):
    return render_template("500.html"), 500

if __name__ == '__main__' :
    app.run(host = '0.0.0.0', port=5020)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5020
 * Running on http://192.168.0.31:5020
Press CTRL+C to quit
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/common.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/common.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/footer.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/prism.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/sweetalert.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/custom.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:28:05] "GET /static/img/diamond.png HTTP/1.1" 200 -
127.0

127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/custom.js HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/img/diamond.png HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/img/webcam.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/img/2022-08-30_dodo_1.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/img/map.png HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/img/tel.png HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /static/img/email.png HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:31] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:36:34] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:36:36] "GET /mypage/cage/n2 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2022 21:36:36] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:36] "GET /static/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [02/Oct/2022 21:36:36] "GET /static/foo